In [ ]:
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path

In [ ]:
CSV_PATH = r"Traffic_Lines_-5158403106151148634.csv"
GEO_PATH = r"Traffic_Lines_-8205076846245197459.geojson"

CSV_ID_COL = "OBJECTID"
GEO_ID_COL = "OBJECTID"

OUT_BASENAME = Path("aadt_davidson")

In [ ]:
CSV_PATH = CSV_PATH if "CSV_PATH" in globals() else "Traffic_Lines_-5158403106151148634.csv"
OUT_BASENAME = OUT_BASENAME if "OUT_BASENAME" in globals() else Path("aadt_davidson")

YEAR_ANY_RE = re.compile(r"(?:^|[^0-9])((?:19|20)\d{2})(?:[^0-9]|$)", flags=re.IGNORECASE)

def _find_long_year_col(df: pd.DataFrame) -> str | None:
    """returns the column that stores the year (prefers name-based, then value-based)"""
    for c in df.columns:
        s = str(c).strip().lower()
        if s in {"year", "yr"} or "year" in s:
            return c
    best, best_share = None, -1.0
    for c in df.columns:
        s = pd.to_numeric(df[c], errors="coerce")
        share = s.between(1990, 2035).mean()
        if share > best_share:
            best, best_share = c, share
    return best if best_share >= 0.50 else None

def _find_long_aadt_col(df: pd.DataFrame, year_col: str | None) -> str | None:
    """returns the column that stores AADT (excludes the detected year column)"""
    def _ok(c):
        return c != year_col
    # prefers explicit names
    for c in df.columns:
        if _ok(c) and "aadt" in str(c).lower() and "year" not in str(c).lower():
            return c
    # falls back to numeric-dense column with range, excluding year_col
    best, best_density, best_range = None, -1.0, -1.0
    for c in df.columns:
        if not _ok(c):
            continue
        v = pd.to_numeric(df[c], errors="coerce")
        dens = v.notna().mean()
        rng = (v.max() - v.min()) if v.notna().any() else -1
        if dens > best_density or (dens == best_density and rng > best_range):
            best, best_density, best_range = c, dens, rng
    return best if best_density >= 0.50 else None

def _detect_wide_year_cols(df: pd.DataFrame) -> list[str]:
    """returns columns that contain a 4-digit year in their name"""
    cols = []
    for c in df.columns:
        if YEAR_ANY_RE.search(str(c)):
            cols.append(c)
    bad_tokens = {"objectid","route id","route_id","gisid","segment","station","count_station"}
    return [c for c in cols if not any(tok in str(c).lower() for tok in bad_tokens)]

def _extract_year_from_name(colname: str) -> int:
    """extracts the 4-digit year from a column name"""
    m = YEAR_ANY_RE.search(str(colname))
    if not m:
        raise ValueError(f"Cannot parse year from column: {colname}")
    return int(m.group(1))

df = pd.read_csv(CSV_PATH, low_memory=False)

year_col = _find_long_year_col(df)
aadt_col = _find_long_aadt_col(df, year_col)
wide_year_cols = _detect_wide_year_cols(df)

print("Detected year_col:", year_col)
print("Detected aadt_col:", aadt_col)
print("Wide year-like columns (sample):", wide_year_cols[:10])

# builds long_df
if year_col is not None and aadt_col is not None:
    # treats file as already long format
    year_s = pd.to_numeric(df[year_col], errors="coerce")
    aadt_s = pd.to_numeric(df[aadt_col], errors="coerce")
    long_df = pd.DataFrame({"year": year_s, "aadt": aadt_s}).dropna(subset=["year","aadt"])
else:
    # treats as wide if possible
    if not wide_year_cols:
        raise ValueError(
            "No suitable AADT column and no year-like header columns found.\n"
            f"Columns (first 25): {list(df.columns)[:25]}"
        )
    melted = df.melt(
        id_vars=[c for c in df.columns if c not in wide_year_cols],
        value_vars=wide_year_cols,
        var_name="year_raw",
        value_name="aadt"
    )
    long_df = melted.assign(
        year=lambda d: d["year_raw"].map(_extract_year_from_name),
        aadt=lambda d: pd.to_numeric(d["aadt"], errors="coerce")
    ).dropna(subset=["year","aadt"])[["year","aadt"]]

# aggregates to calendar-year totals
traffic_year = (
    long_df.groupby("year", as_index=False)["aadt"].sum()
           .rename(columns={"aadt":"aadt_total"})
           .sort_values("year")
           .reset_index(drop=True)
)

out_dir = Path("outputs"); out_dir.mkdir(exist_ok=True)
tidy_path = out_dir / "aadt_davidson_long.csv"
year_path = out_dir / "aadt_davidson_year_totals.csv"
long_df[["year","aadt"]].to_csv(tidy_path, index=False)
traffic_year.to_csv(year_path, index=False)

print("Years present:", traffic_year["year"].astype(int).tolist())
display(traffic_year.head(10)); display(traffic_year.tail(10))

In [ ]:
# === Coverage check: verifies needed years for ACS alignment (2013–2023) ===
need = list(range(2013, 2024))
have = set(traffic_year["year"].astype(int))
missing = sorted([y for y in need if y not in have])

print("Years present:", sorted(have))
print("Years missing (need 2013–2023):", missing)

# writes a small report
import pandas as pd, numpy as np
pd.DataFrame({"year": need, "present": [y in have for y in need]}) \
  .to_csv(Path("outputs") / "aadt_needed_years_2013_2023.csv", index=False)
print("Wrote:", Path("outputs") / "aadt_needed_years_2013_2023.csv")

In [ ]:
CANDIDATE_ID_COLS = [
    "STATION","STATION_ID","STATIONID","COUNT_STATION","COUNT_STATION_ID",
    "LOC_ID","LOCATION_ID","SITE_NO","SITENO","POINT_ID","POINTID",
    "SEGMENT_ID","SEGMENTID","GISID","GIS_ID","ID","OBJECTID"
]

def _auto_id(cols):
    low = {c.lower(): c for c in cols}
    for cand in CANDIDATE_ID_COLS:
        if cand.lower() in low:
            return low[cand.lower()]
    return None

def _norm_id(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.str.replace(r"\s+", "", regex=True)
    s = s.str.upper()
    s = s.str.replace(r"^STATION[_\-]?", "", regex=True)
    return s

In [ ]:
YEAR_COL_RE = re.compile(r"^(?:AADT[_\-\s]*)?(19|20)\d{2}$")

def _detect_year_cols(df: pd.DataFrame):
    yc = [c for c in df.columns if YEAR_COL_RE.match(str(c).strip())]
    if not yc:
        yc = [c for c in df.columns if re.fullmatch(r"(19|20)\d{2}", str(c).strip())]
    return yc

def _year_from(colname: str) -> int:
    m = re.search(r"(19|20)\d{2}", str(colname))
    if not m:
        raise ValueError(f"Could not parse year from column: {colname}")
    return int(m.group(0))

In [ ]:
def _tidy_from_wide(df: pd.DataFrame, id_col: str, keep_cols=None) -> pd.DataFrame:
    keep_cols = keep_cols or []
    year_cols = _detect_year_cols(df)

    # Tall format (e.g., 'AADT Year' + 'AADT')
    if not year_cols:
        year_candidates = [c for c in df.columns if "year" in str(c).lower()]
        year_col = next((c for c in ["year","Year"] + year_candidates if c in df.columns), None)
        aadt_col = next((c for c in ["AADT","aadt","AADT_TOTAL","TOTAL_AADT","value"] if c in df.columns), None)
        if not (year_col and aadt_col):
            raise ValueError("Need a year-like column and an AADT column.")
        out = df[[id_col, year_col, aadt_col] + [c for c in keep_cols if c in df.columns]].copy()
        out = out.rename(columns={year_col: "year", aadt_col: "aadt"})
        out["year"] = pd.to_numeric(out["year"], errors="coerce").astype("Int64")
        out["aadt"] = pd.to_numeric(out["aadt"], errors="coerce")
        return out

    # Wide format (year columns across)
    long_df = df.melt(
        id_vars=[c for c in df.columns if c not in year_cols],
        value_vars=year_cols, var_name="year_raw", value_name="aadt"
    )
    long_df["year"] = long_df["year_raw"].apply(_year_from)
    long_df = long_df.drop(columns=["year_raw"])
    keep = [id_col, "year", "aadt"] + [c for c in keep_cols if c in long_df.columns]
    long_df = long_df[keep].copy()
    long_df["aadt"] = pd.to_numeric(long_df["aadt"], errors="coerce")
    return long_df

In [ ]:
df_head = pd.read_csv(CSV_PATH, nrows=5)
print("CSV columns:", sorted(df_head.columns.tolist()))
csv_ids = [c for c in df_head.columns if c.upper() in CANDIDATE_ID_COLS]
print("Likely CSV IDs:", csv_ids)

gdf_head = gpd.read_file(GEO_PATH).head(5)
print("GEO columns:", sorted(gdf_head.columns.tolist()))
geo_ids = [c for c in gdf_head.columns if c.upper() in CANDIDATE_ID_COLS]
print("Likely GEO IDs:", geo_ids)

In [ ]:
df = pd.read_csv(CSV_PATH)
df[CSV_ID_COL] = _norm_id(df[CSV_ID_COL])

keep_cols = [c for c in ["Route ID","ROUTE","ROAD_NAME","ROUTE_NAME","ROADWAY","DIR","DIRECTION"] if c in df.columns]
tidy = (_tidy_from_wide(df, id_col=CSV_ID_COL, keep_cols=keep_cols)
        .rename(columns={CSV_ID_COL: "station_id"}))
tidy = tidy[tidy["aadt"].notna()].copy()

OUT_BASENAME.parent.mkdir(parents=True, exist_ok=True)
tidy_out = OUT_BASENAME.with_suffix(".clean.csv")
tidy.to_csv(tidy_out, index=False)

print("Saved:", tidy_out, "| rows:", len(tidy))
tidy.head()


In [ ]:
needed_ids = set(tidy["station_id"].unique())

gdf = gpd.read_file(GEO_PATH)
gdf = gdf[[GEO_ID_COL, "geometry"]]  # drop extras early
gdf[GEO_ID_COL] = _norm_id(gdf[GEO_ID_COL])
gdf = gdf[gdf[GEO_ID_COL].isin(needed_ids)].drop_duplicates(subset=[GEO_ID_COL])

print("Geometry rows after filter:", len(gdf))

In [ ]:
if gdf.crs is None:
    gdf = gdf.set_crs(epsg=4326)
elif getattr(gdf.crs, "to_epsg", lambda: None)() != 4326:
    gdf = gdf.to_crs(epsg=4326)

gdf = gdf.rename(columns={GEO_ID_COL: "station_id"})
merged = gdf.merge(tidy, on="station_id", how="inner", sort=False)

print("Merged rows:", len(merged))
merged.head(3).drop(columns="geometry")

In [ ]:
out_geojson = OUT_BASENAME.with_suffix(".geojson")
merged.to_file(out_geojson, driver="GeoJSON")
print("Wrote:", out_geojson)


In [ ]:
year_counts = tidy.groupby("year").size().rename("rows").reset_index().sort_values("year")
year_counts


In [ ]:
tidy.sort_values("aadt", ascending=False).head(10)


In [ ]:
bad_years = tidy[~tidy["year"].between(1990, 2025)]["year"].unique()
print("Out-of-range years:", bad_years)

In [ ]:
tidy["year"] = tidy["year"].astype(int)

# Now group and plot
totals = tidy.groupby("year", as_index=False)["aadt"].sum()

plt.figure(figsize=(8,4))
plt.plot(totals["year"], totals["aadt"], marker="o")
plt.title("Total AADT across stations by year")
plt.xlabel("Year")
plt.ylabel("Sum of AADT")
plt.xticks(totals["year"])  # show only the years present
plt.tight_layout()
#plt.show()

In [ ]:
# Keep only plausible years (e.g., 1990–2025)
totals = (
    tidy[tidy["year"].between(1990, 2025)]
    .groupby("year", as_index=False)["aadt"]
    .sum()
)

plt.figure(figsize=(8,4))
plt.plot(totals["year"], totals["aadt"], marker="o")
plt.title("Total AADT across stations by year")
plt.xlabel("Year")
plt.ylabel("Sum of AADT")
plt.tight_layout()
#plt.show()


In [ ]:
top_station = tidy["station_id"].value_counts().idxmax()
s = tidy[tidy["station_id"] == top_station].sort_values("year")

plt.figure(figsize=(8,4))
plt.plot(s["year"], s["aadt"])
plt.title(f"AADT trend — station {top_station}")
plt.xlabel("Year"); plt.ylabel("AADT")
plt.tight_layout(); #plt.show()


In [ ]:
# Required fields & aliases
fields = ["station_id", "year", "aadt"]
aliases = ["Station", "Year", "AADT"]

# Optional fields (check for existence in merged)
optional = [
    ("Route ID", "Route ID"),
    ("ROUTE", "ROUTE"),
    ("ROAD_NAME", "Road Name"),
    ("ROUTE_NAME", "Route Name"),
    ("ROADWAY", "Roadway")
]

for col, alias in optional:
    if col in merged.columns:
        fields.append(col)
        aliases.append(alias)

# Now both lists match in length
m = folium.Map(location=[36.1627, -86.7816], zoom_start=10)
folium.GeoJson(
    merged.to_json(),
    name="AADT",
    tooltip=folium.features.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        sticky=False
    ),
).add_to(m)
folium.LayerControl().add_to(m)
m


In [ ]:
tidy["year"] = tidy["year"].astype(int)
print("Unique years in tidy:", sorted(tidy["year"].unique()))

In [ ]:
topN = 25  # change as needed
top_segments = (
    tidy.sort_values("aadt", ascending=False)
        .head(topN)
        .copy()
)

top_segments[["station_id","year","aadt"] + [c for c in ["Route ID","ROUTE","ROAD_NAME","ROUTE_NAME","ROADWAY"] if c in top_segments.columns]]


In [ ]:
plt.figure(figsize=(8,10))
plt.barh(
    top_segments["station_id"].astype(str)[::-1],
    top_segments["aadt"][::-1]
)
plt.title("Top AADT Segments — 2024")
plt.xlabel("AADT")
plt.ylabel("Station ID")
plt.tight_layout()
plt.show()


In [ ]:
route_cols = [c for c in ["Route ID","ROUTE","ROUTE_NAME","ROAD_NAME","ROADWAY"] if c in tidy.columns]
agg_cols = ["aadt"]
if "VMT" in tidy.columns:
    agg_cols.append("VMT")

by_route = (
    tidy.groupby(route_cols, dropna=False)[agg_cols]
        .sum(numeric_only=True)
        .reset_index()
        .sort_values("aadt", ascending=False)
)

by_route.head(15)


In [ ]:
# If you've already created `merged`, it’s 2024-only.
# If not, run your earlier merge steps first.
print("Merged rows (2024 only):", len(merged))


In [ ]:
# Required tooltip fields
fields = ["station_id", "aadt"]
aliases = ["Station", "AADT"]

# Optional details if present
optional = [
    ("Route ID", "Route ID"),
    ("ROUTE", "ROUTE"),
    ("ROAD_NAME", "Road Name"),
    ("ROUTE_NAME", "Route Name"),
    ("ROADWAY", "Roadway"),
    ("VMT", "VMT")
]
for col, alias in optional:
    if col in merged.columns:
        fields.append(col)
        aliases.append(alias)

m = folium.Map(location=[36.1627, -86.7816], zoom_start=10)

# Simple style by quantiles of AADT
try:
    q = merged["aadt"].quantile([0, 0.5, 0.75, 0.9, 0.99, 1]).tolist()
    def style_fn(feature):
        a = feature["properties"].get("aadt", 0) or 0
        if a <= q[1]: w = 2
        elif a <= q[2]: w = 3
        elif a <= q[3]: w = 4
        elif a <= q[4]: w = 5
        else: w = 6
        return {"weight": w}
except Exception:
    def style_fn(_): return {"weight": 3}

folium.GeoJson(
    merged.to_json(),
    name="AADT 2024",
    style_function=style_fn,
    tooltip=folium.features.GeoJsonTooltip(fields=fields, aliases=aliases, sticky=False),
).add_to(m)

folium.LayerControl().add_to(m)
m


In [ ]:
# Segment-level 2024
seg2024 = merged.drop(columns="geometry")
seg2024_out = OUT_BASENAME.with_suffix(".segments_2024.csv")
seg2024.to_csv(seg2024_out, index=False)
print("Wrote:", seg2024_out)

# Route-level 2024 (from by_route above)
route2024_out = OUT_BASENAME.with_suffix(".routes_2024.csv")
by_route.to_csv(route2024_out, index=False)
print("Wrote:", route2024_out)


In [ ]:
# reuses CSV_PATH and OUT_BASENAME if already defined; otherwise sets defaults
CSV_PATH = CSV_PATH if "CSV_PATH" in globals() else "Traffic_Lines_-5158403106151148634.csv"
OUT_BASENAME = OUT_BASENAME if "OUT_BASENAME" in globals() else Path("aadt_davidson")

# defines year-column detector if missing
if "_detect_year_cols" not in globals():
    YEAR_COL_RE = re.compile(r"^(?:AADT[_\-\s]*)?(19|20)\d{2}$")
    def _detect_year_cols(df: pd.DataFrame):
        yc = [c for c in df.columns if YEAR_COL_RE.match(str(c).strip())]
        if not yc:
            yc = [c for c in df.columns if re.fullmatch(r"(19|20)\d{2}", str(c).strip())]
        return yc
    def _year_from(colname: str) -> int:
        m = re.search(r"(19|20)\d{2}", str(colname))
        if not m:
            raise ValueError(f"Could not parse year from column: {colname}")
        return int(m.group(0))

df_head = pd.read_csv(CSV_PATH, nrows=5, low_memory=False)
print("CSV columns (sample):", sorted(df_head.columns.tolist())[:25])

df = pd.read_csv(CSV_PATH, low_memory=False)
year_cols = _detect_year_cols(df)
years_parsed = sorted({_year_from(c) for c in year_cols})

print("\nDetected year columns:", year_cols[:20], ("..." if len(year_cols) > 20 else ""))
print("Parsed years:", years_parsed)

# builds year coverage summary
summary = []
for c in year_cols:
    y = _year_from(c)
    s = pd.to_numeric(df[c], errors="coerce")
    summary.append({
        "year": y,
        "non_null_rows": int(s.notna().sum()),
        "min_aadt": float(np.nanmin(s)) if s.notna().any() else np.nan,
        "median_aadt": float(np.nanmedian(s)) if s.notna().any() else np.nan,
        "max_aadt": float(np.nanmax(s)) if s.notna().any() else np.nan
    })
years_summary = pd.DataFrame(summary).sort_values("year")
display(years_summary)

OUT_BASENAME.parent.mkdir(parents=True, exist_ok=True)
years_summary.to_csv(OUT_BASENAME.with_suffix(".years_summary.csv"), index=False)
print("Wrote:", OUT_BASENAME.with_suffix(".years_summary.csv"))

In [ ]:
def _tidy_from_wide(df: pd.DataFrame, id_col: str | None = None, keep_cols=None) -> pd.DataFrame:
    """melts wide year columns into long format and extracts numeric year"""
    keep_cols = keep_cols or []
    ycols = _detect_year_cols(df)
    if not ycols:
        raise ValueError("No year-like columns found.")
    long_df = df.melt(
        id_vars=[c for c in df.columns if c not in ycols],
        value_vars=ycols, var_name="year_raw", value_name="aadt"
    )
    long_df["year"] = long_df["year_raw"].map(lambda s: _year_from(s))
    cols = (["year", "aadt"] + [c for c in keep_cols if c in long_df.columns])
    out = long_df[cols].copy()
    out["aadt"] = pd.to_numeric(out["aadt"], errors="coerce")
    return out

# chooses optional keep columns if present
keep_cols = [c for c in ["Route ID","ROUTE","ROAD_NAME","ROUTE_NAME","ROADWAY","DIR","DIRECTION","COUNTY","COUNTY_NAME"] if c in df.columns]
tidy = _tidy_from_wide(df, keep_cols=keep_cols).dropna(subset=["aadt"])

# aggregates to year; filters to target window if desired
traffic_year = (
    tidy.groupby("year", as_index=False)["aadt"].sum()
        .rename(columns={"aadt": "aadt_total"})
        .sort_values("year")
)
display(traffic_year.head(10))
display(traffic_year.tail(10))

# writes canonical tidy and aggregated outputs for Capstone3
out_dir = Path("outputs"); out_dir.mkdir(exist_ok=True)
tidy_path = out_dir / "aadt_davidson_long.csv"
year_path = out_dir / "aadt_davidson_year_totals.csv"

tidy.to_csv(tidy_path, index=False)
traffic_year.to_csv(year_path, index=False)
print("Wrote:", tidy_path)
print("Wrote:", year_path)

In [ ]:
# === 2013–2023 readiness check: verifies coverage needed for ACS 5-year periods ===
need_years = list(range(2013, 2024))  # inclusive 2013..2023
have_years = set(map(int, traffic_year["year"].tolist()))
missing = sorted([y for y in need_years if y not in have_years])

print("Years needed (2013–2023):", need_years)
print("Years present:", sorted(have_years))
print("Years missing:", missing)

# saves a small report
pd.DataFrame({"need_year": need_years, "present": [y in have_years for y in need_years]}) \
  .to_csv(Path("outputs") / "aadt_needed_years_2013_2023.csv", index=False)
print("Wrote:", Path("outputs") / "aadt_needed_years_2013_2023.csv")